In [156]:
from math import exp
from random import uniform

In [113]:
def sigmoid(x):
    return 1 / (1 + exp(-x))

In [114]:
def get_deltaW(grad, last_deltaW):
    E = 0.7 #speed of learning

    A = 0.3 #moment
    
    return E * grad + A * last_deltaW

In [166]:
class InputNeuro:
    
    def __init__(self, value, weights):
        self.__value = value
        self.__weights = weights
        self.__deltaW = [0]
        
    def __repr__(self):
        return '''
        Value: {},
        Weights: {},
        Deltas W: 
        '''.format(self.__value, self.__weights, self.__deltaW)
    
    def get_value(self):
        return self.__value
    
    def get_weight(self, hidden_neuro_index):
        return self.__weights[hidden_neuro_index]
    
    def set_weight(self, hidden_neuro_index, weight):
        self.__weights[hidden_neuro_index] = weight
    
    def get_last_deltaW(self):
        return self.__deltaW[-1]
        
    def add_new_deltaW(self, deltaW):
        self.__deltaW.append(deltaW)


In [116]:
class HiddenNeuro:

    def __init__(self, index, neurons, weight, activation_func):
        self.index = index
        self.neurons = neurons
        self.__weight = weight
        self.activation_func = activation_func
        self.__delta = 0
        self.__deltaW = [0]
    
    def get_value(self):
        input_sum = self.__get_sum()
        return self.activation_func(input_sum)
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, weight):
        self.__weight = weight
    
    def get_delta(self):
        return self.__delta
    
    def set_delta(self, delta):
        self.__delta = delta
        
    def get_last_deltaW(self):
        return self.__deltaW[-1]
        
    def add_new_deltaW(self, deltaW):
        self.__deltaW.append(deltaW)
    
    def __get_sum(self):
        input_sum = 0
        for neuro in self.neurons:
            input_sum += neuro.get_value() * neuro.get_weight(self.index)
        return input_sum

In [143]:
class OutputNeuro:
    
    def __init__(self, neurons, activation_func):
        self.neurons = neurons
        self.activation_func = activation_func
        self.__delta = 0
        self.__deltaW = [0]
        
    def get_value(self):
        input_sum = self.__get_sum()
        return self.activation_func(input_sum)
    
    def __get_sum(self):
        input_sum = 0
        for neuro in self.neurons:
            input_sum += neuro.get_value() * neuro.get_weight()
        return input_sum
    
    def get_delta(self):
        return self.__delta
    
    def set_delta(self, delta):
        self.__delta = delta
    
    def get_last_deltaW(self):
        return self.__deltaW[-1]
    
    def add_new_deltaW(self, deltaW):
        self.__deltaW.append(deltaW)
        
    def get_error(self, correct_answer):
        return (self.get_value() - correct_answer) ** 2

In [153]:
input_neurons = [InputNeuro(1, [0.45, 0.78]), InputNeuro(0, [-0.12, 0.13])]
hidden_neurons = [
    HiddenNeuro(0, input_neurons, 1.5, sigmoid),
    HiddenNeuro(1, input_neurons, -2.3, sigmoid)           
                 ]
output_neuron = OutputNeuro(hidden_neurons, sigmoid)

In [154]:
while output_neuron.get_error(answer) > accuracy ** 2:
    output_neuron.set_delta((answer - on1.value()) * ((answer - on1.value()) * on1.value()))
    for hidden_neuron in hidden_neurons:
        new_delta = ((1 - hidden_neuron.get_value()) * hidden_neuron.get_value()) * (hidden_neuron.get_weight() * output_neuron.get_delta())
        hidden_neuron.set_delta(new_delta)
        grad = output_neuron.get_delta() * hidden_neuron.get_value()
        deltaW = get_deltaW(grad, hidden_neuron.get_last_deltaW())
        hidden_neuron.set_weight(hidden_neuron.get_weight() + deltaW)
        hidden_neuron.add_new_deltaW(deltaW)
    for input_neuron in input_neurons:
        for idx, hidden_neuron in enumerate(hidden_neurons):
            deltaH = hidden_neuron.get_delta()
            grad = input_neuron.get_value() * deltaH
            deltaW = get_deltaW(grad, input_neuron.get_last_deltaW())
            input_neuron.set_weight(idx, input_neuron.get_weight(idx) + deltaW)
            input_neuron.add_new_deltaW(deltaW)

In [155]:
output_neuron.get_value()

0.9999138197968985

In [171]:
def train_neural_network(input_values, answer, accuracy=0.0001, hidden_neurons_count=2):
    # init with random weights
    input_neurons = [InputNeuro(value, [uniform(-0.5, 0.5) for _ in range(hidden_neurons_count)]) for value in input_values]
    hidden_neurons = [HiddenNeuro(i, input_neurons, uniform(-0.5, 0.5), sigmoid) for i in range(hidden_neurons_count)]
    output_neuron = OutputNeuro(hidden_neurons, sigmoid)
    era = 1
    
    #train
    while output_neuron.get_error(answer) > accuracy ** 2:
        output_neuron.set_delta((answer - on1.value()) * ((answer - on1.value()) * on1.value()))
        for hidden_neuron in hidden_neurons:
            new_delta = ((1 - hidden_neuron.get_value()) * hidden_neuron.get_value()) * (hidden_neuron.get_weight() * output_neuron.get_delta())
            hidden_neuron.set_delta(new_delta)
            grad = output_neuron.get_delta() * hidden_neuron.get_value()
            deltaW = get_deltaW(grad, hidden_neuron.get_last_deltaW())
            hidden_neuron.set_weight(hidden_neuron.get_weight() + deltaW)
            hidden_neuron.add_new_deltaW(deltaW)
        for input_neuron in input_neurons:
            for idx, hidden_neuron in enumerate(hidden_neurons):
                deltaH = hidden_neuron.get_delta()
                grad = input_neuron.get_value() * deltaH
                deltaW = get_deltaW(grad, input_neuron.get_last_deltaW())
                input_neuron.set_weight(idx, input_neuron.get_weight(idx) + deltaW)
                input_neuron.add_new_deltaW(deltaW)
        era += 1
    print('Answer: ', output_neuron.get_value())
    print('Error: ', output_neuron.get_error(answer))
    print('Eras: ', era)

In [173]:
train_neural_network([0, 1], 1, 0.001, 3)

Answer:  0.9991952055834477
Error:  6.476940529138306e-07
Eras:  31
